# 第27章 更多实例

- Python的类系统实际上很大程度上就是在一堆对象中查找属性，并为函数给定一个特殊的第一个参数

## 步骤1：创建实例

- 类的方法第一个参数名实际上代表的是新创建的实例对象
- 类的方法第一个参数名不一定非得是`self`，只是约定俗成而已

In [1]:
class Test:
    def f1(ins, name):
        ins.name = name
        print "set the name"

In [2]:
test = Test()
test.f1("Bob")

set the name


In [3]:
test.name

'Bob'

- `__init__`函数会在每次创建新的实例对象时自动调用

## 步骤3：运算符重载

- 打印一个对象，会显示对象的`__str__`方法所返回的内容，要么自己定义一个该方法，要么从一个积累继承一个该方法

## 步骤4：通过子类定制行为

- 如下的常规方法调用：`instance.method(args...)`，由Python自动地转换为如下的同等形式：`class.method(instance, args...)`

## 步骤5：定制构造函数

In [4]:
class Person:
    def __init__(self, name, job=None, pay=0):
        self.name = name
        self.job = job
        self.pay = pay
    def lastName(self):
        return self.name.split()[-1]
    def giveRaise(self, percent):
        self.pay = int(self.pay * (1 + percent))
    def __str__(self):
        return '[Person: %s, %s]' % (self.name, self.pay)

class Manager(Person):
    def __init__(self, name, pay):
        Person.__init__(self, name, 'mgr', pay)
    def giveRaise(self, percent, bonus=.10):
        Person.giveRaise(self, percent + bonus)
        
bob = Person('Bob Smith')
sue = Person('Sue Jones', job='dev', pay=100000)
print bob
print sue
print bob.lastName(), sue.lastName()
sue.giveRaise(.10)
print sue
tom = Manager('Tom Jones', 50000)
tom.giveRaise(.10)
print tom.lastName()
print tom

[Person: Bob Smith, 0]
[Person: Sue Jones, 100000]
Smith Jones
[Person: Sue Jones, 110000]
Jones
[Person: Tom Jones, 60000]


## 步骤6：使用内省工具

- 内置的`instance.__class__`属性提供了一个从实例到创建它的类的链接，类反过来有一个`__name__`，还有一个`__bases__`序列，提供了基类的访问
- 内置的`object.__dict__`属性提供了一个字典，带有一个键/值对，以便每个属性都附加到一个命名空间对象

In [5]:
Manager.__name__

'Manager'

In [6]:
Manager.__bases__

(<class __main__.Person at 0x10b74a258>,)

In [7]:
bob = Person('Bob Smith')
print bob

[Person: Bob Smith, 0]


In [8]:
bob.__class__

<class __main__.Person at 0x10b74a258>

In [9]:
bob.__class__.__name__

'Person'

In [10]:
list(bob.__dict__.keys())

['pay', 'job', 'name']

### 一种通用显示工具

In [11]:
class AttrDisplay:
    def gatherAttrs(self):
        attrs = []
        for key in sorted(self.__dict__):
            attrs.append('%s=%s' % (key, getattr(self, key)))
        return ', '.join(attrs)
    def __str__(self):
        return '[%s: %s]' % (self.__class__.__name__, self.gatherAttrs())

class TopTest(AttrDisplay):
    count = 0
    def __init__(self):
        self.attr1 = TopTest.count
        self.attr2 = TopTest.count + 1
        TopTest.count += 2
class SubTest(TopTest):
    pass

X, Y = TopTest(), SubTest()
print X
print Y

[TopTest: attr1=0, attr2=1]
[SubTest: attr1=2, attr2=3]


### 实例与类属性的关系

- `__dict__`只会获取实例本身的属性，`dir()`还可以获取实例从其树上面的类那里继承来的属性

In [12]:
AttrDisplay.__dict__.keys()

['gatherAttrs', '__module__', '__str__', '__doc__']

In [13]:
TopTest.__dict__.keys()

['count', '__module__', '__doc__', '__init__']

In [14]:
X.__dict__.keys()

['attr2', 'attr1']

In [15]:
dir(X)

['__doc__',
 '__init__',
 '__module__',
 '__str__',
 'attr1',
 'attr2',
 'count',
 'gatherAttrs']

### 工具类的命名考虑

- 为了减少名称冲突的机会，Python常常对于不想做其它用途的方法添加一个单下划线的前缀，例如之前的`gatherAttrs`可以变为`_gatherAttrs`
- 一种更好的但不太常用的方法是，只在方法名前面使用双下划线，如`__gatherAttrs`，Python自动扩展这样的名称，以包含类的名称，从而使它们变得真正唯一，这一功能通常叫做伪私有类属性（后面我们还会详细讲到）